In [1]:
# Standard libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path1p, "lib")
pwd_mat = os.path.join(os.path.join(path2p, "data/"), "sim_ds_mat")
pwd_h5 = os.path.join(os.path.join(path2p, "data/"), "sim_ds_h5")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from matlab.matlab_yaro_lib import read_mat
from signal_lib import downsample
from corr_lib import sprMat
from idtxl_wrapper import idtxlParallelCPUMulti, idtxlResultsParse
from qt_wrapper import gui_fpath

# IDTxl libraries
from idtxl.bivariate_mi import BivariateMI
from idtxl.multivariate_mi import MultivariateMI
from idtxl.bivariate_te import BivariateTE
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from idtxl.visualise_graph import plot_network

Appended library directory /home/ubuntu/work/nest-mesoscopic-metric-test/codes/lib


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [2]:
# Declare constants
params = {
    "exp_timestep" : 0.05, # 50ms, the standard measurement interval
    "bin_timestep" : 0.2,  # 200ms, the binned interval
    
    # The standard timesteps for different scenarios
    "range_CUE" : (1.0, 1.5),    # 1-1.5 seconds trial time
    "range_TEX" : (2.0, 3.5),    # 2-3.5 seconds trial time
    "range_LIK" : (3.5, 6.0),    # 3.5-6 seconds trial time
    "range_ALL" : (0.0, 10.0),   # 0-10  seconds

#     "samples_window" : "ALL",
    "trial_types"    : ["iGO", "iNOGO"],
    "subsampling"    : True,   # To bin or not to bin data
#     "subsampling_windows_lenghts" : 4*np.ones((1,51)),  #uniform subsampling, 50 bins
    
    "te_delay_min" : 1,
    "te_delay_max" : 5
    
    
    
#     "te_delay_stepsize" : 1,
#     "te_receiver_embedding_tau" : 1,
#     "te_sampling_rate" : 20
}

# GUI: Select videos for training
pwd_tmp = "./"
next_path = "AAAAA"
params['pwd_data_folders'] = []
while next_path != '':
    print("Currently using data folders:", params['pwd_data_folders'])
    next_path = gui_fpath("Select one more data directory, or cancel", pwd_tmp)
    if next_path != '':
        params['pwd_data_folders'] += [next_path]
        pwd_tmp = os.path.dirname(next_path)  # Next time choose from parent folder
print('Done')

Currently using data folders: []
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_20_a']
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_20_a', '/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_21_a']
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_20_a', '/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_21_a', '/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_22_a']
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_20_a', '/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_21_a', '/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_22_a', '/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_23_a']
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_20_a', '/mnt/hif

## Swipe Transfer Entropy via IDTxl

Notation of IDTxl
* `(processes, samples, replications)` equivalent to ours `(channels, times, trials)`
* Thus Yaro matrix `(nTrials, nTimes, nChannels)` is given by `'rsp'`, but
* For example `(nChannels, nTimes)` would be `'ps'`

In [ ]:
%%time
for folderPathName in params['pwd_data_folders']:
    folderName = os.path.basename(folderPathName)
    
    #############################
    # Reading and downsampling
    #############################

    # Read LVM file from command line
    data, behaviour = read_mat(folderPathName)

    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

    assert nTimes == 201, "The number of timesteps must be 201 for all data, got "+str(nTimes)
    nTimesDownsampled = int(nTimes * params["exp_timestep"] / params["bin_timestep"])
    print(nTimesDownsampled)

    # Timeline (x-axis)
    tlst = params["exp_timestep"] * np.linspace(0, nTimes, nTimes)

    # Downsample data
    print("Downsampling from", params["exp_timestep"], "ms to", params["bin_timestep"], "ms")
    if params["subsampling"]:
        params["timestep"] = params["bin_timestep"]
        tlst_new = downsample(tlst, data[0, :, 0], nTimesDownsampled)[0]
        data_new = np.array([[downsample(tlst, data[i, :, j], nTimesDownsampled)[1] 
                              for i in range(nTrials)] 
                             for j in range(nChannels)])
    else:
        params["timestep"] = params["exp_timestep"]

#     # Plot downsampling for comparison
#     plt.figure()
#     plt.plot(tlst, data[0, :, 0], label='original')
#     plt.plot(tlst_new, data_new[0, 0, :], '.-', label='downsampled')
#     plt.title("Example before and after downsampling")
#     plt.legend()
#     plt.show()

    # Replace old data with subsampled one    
    tlst, data = tlst_new, data_new.transpose(1, 2, 0)
    nTrials, nTimes, nChannels = data.shape
    print("After downsampling data shape is (nTrials, nTimes, nChannels)=", data.shape)
    
    for trialType in params['trial_types']:
        
        if trialType is None:
            dataEff = data
            fileNameSuffix = ""
        else:
            dataEff = data[behaviour[trialType][0] - 1]
            fileNameSuffix = "_" + trialType
            print("For trialType =", trialType, "the shape is (nTrials, nTimes, nChannels)=", dataEff.shape)

        #############################
        # Analysis
        #############################

        idtxl_settings = {
            'dim_order'       : 'rsp',
            'methods'         : ["BivariateTE", "MultivariateTE"],
            'cmi_estimator'   : 'JidtGaussianCMI',
            'max_lag_sources' : params["te_delay_max"],
            'min_lag_sources' : params["te_delay_min"]
       }

        teWindow = params["te_delay_max"] + 1

        data_range = list(range(nTimes - teWindow))
        data_lst = [dataEff[:, i:i + teWindow, :] for i in data_range]
        rez = idtxlParallelCPUMulti(data_lst, idtxl_settings, folderName)

        for iMethod, method in enumerate(idtxl_settings['methods']):
            
            teSwipeMat  = np.zeros((nChannels, nChannels, nTimes)) + np.nan
            lagSwipeMat = np.zeros((nChannels, nChannels, nTimes)) + np.nan
            pSwipeMat   = np.zeros((nChannels, nChannels, nTimes)) + np.nan
            
            for iRange in data_range:
                te_mat, lag_mat, p_mat = idtxlResultsParse(rez[iMethod][iRange], nChannels, method=method, storage='matrix')

                teSwipeMat[:,:, iRange + params["te_delay_max"]]  = te_mat
                lagSwipeMat[:,:, iRange + params["te_delay_max"]] = lag_mat
                pSwipeMat[:,:, iRange + params["te_delay_max"]]   = p_mat

            #######################
            # Save results to file
            #######################
            savename = os.path.join(pwd_h5, folderName + fileNameSuffix + '_' + method + '_swipe' + '.h5')
            print(savename)

            h5f = h5py.File(savename, "w")

            grp_rez = h5f.create_group("results")
            grp_rez['TE_table'] = teSwipeMat
            grp_rez['delay_table'] = lagSwipeMat
            grp_rez['p_table'] = pSwipeMat

            h5f.close()

Reading Yaro data from /mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_20_a
Loaded neuronal data with (nTrials, nTimes, nChannels)= (682, 201, 48)
50
Downsampling from 0.05 ms to 0.2 ms
